In [27]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import time
import os
import asyncio, aiohttp, nest_asyncio
from aiohttp_retry import RetryClient, ExponentialRetry
nest_asyncio.apply()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

trailing_num_days = 90

chains_tg = [
        'Ethereum'
        ,'Optimism'
        ,'Binance'
        ,'Arbitrum'
        ,'Polygon'
        ,'Avalanche'
        ]

In [28]:
#https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library

import logging
import requests

from requests.adapters import HTTPAdapter, Retry

# logging.basicConfig(level=logging.DEBUG)

s = requests.Session()
retries = Retry(total=10, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

pwd = os.getcwd()
if 'L2 TVL' in pwd:
    prepend = ''
else:
    prepend = 'L2 TVL/'

In [29]:
all_api = 'https://api.llama.fi/charts/'
dflist = []
for i in chains_tg:
        print(i)
        res = pd.DataFrame( r.get(all_api + str(i), headers=header).json() )
        res['chain'] = str(i)
        res['date'] = pd.to_datetime(res['date'], unit ='s') #convert to days
        res['rank_desc'] = res['date'].rank(method='dense',ascending=False).astype(int)
        res = res.sort_values(by='date',ascending=True)
        res['t90d_totalLiquidityUSD'] = res['totalLiquidityUSD'].shift(90)
        dflist.append(res)
dfs = pd.concat(dflist)

Ethereum
Optimism
Binance
Arbitrum
Polygon
Avalanche


In [30]:
current_df = dfs[dfs['rank_desc'] == 1]
current_df['current_tvl_share'] = current_df['totalLiquidityUSD'] / (current_df['totalLiquidityUSD'].sum())
current_df['prev_tvl_share'] = current_df['t90d_totalLiquidityUSD'] / (current_df['t90d_totalLiquidityUSD'].sum())
display(current_df)
current_df.to_csv('tvl_comps.csv')

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_32965/3851270728.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_df['current_tvl_share'] = current_df['totalLiquidityUSD'] / (current_df['totalLiquidityUSD'].sum())
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_32965/3851270728.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_df['prev_tvl_share'] = current_df['t90d_totalLiquidityUSD'] / (current_df['t90d_totalLiquidityUSD'].sum())


,date,totalLiquidityUSD,chain,rank_desc,t90d_totalLiquidityUSD,current_tvl_share,prev_tvl_share
1384,2022-10-19,4.641671e+10,Ethereum,1,4.685844e+10,0.800821,0.792217
462,2022-10-19,1.019631e+09,Optimism,1,3.437586e+08,0.017592,0.005812
718,2022-10-19,6.467787e+09,Binance,1,6.519857e+09,0.111588,0.110229
750,2022-10-19,1.029876e+09,Arbitrum,1,6.696644e+08,0.017768,0.011322
740,2022-10-19,1.366622e+09,Polygon,1,1.753773e+09,0.023578,0.029650
624,2022-10-19,1.660768e+09,Avalanche,1,3.002972e+09,0.028653,0.050770
